# 导入数据

In [7]:
from yz.task_dec2high_pddl import *

In [8]:
args = set_up_args()

In [9]:
args.gpu = True

In [10]:
task2plan_train = load_task_and_plan_json(args, "train")

{'tests_seen': 1533, 'tests_unseen': 1529, 'train': 21023, 'valid_seen': 820, 'valid_unseen': 821}


In [11]:
tpd = TaskPlanDataset(task2plan_train)

# 写入字典

In [12]:
#tpd[np.random.randint(1000)]

In [13]:
#tpd.write_vocab("yz/vocab.txt")

# 训练

In [14]:
train_dataloader = DataLoader(tpd, batch_size=args.batch, shuffle=True,)

In [15]:
tasks, plans = next(iter(train_dataloader))

In [16]:
tasks

('place two bars of soap in the garbage bin to the right of the toilet turn around to face the center of the bathroom sink counter. grab a soap bar off of the bathroom counter. turn left and walk to the end of the room where the toilet is, then turn to face the toilet. place the bar of soap in the garbage bin to the right of the toilet. move to the left a bit and face the toilet again. grab the bar of soap off of the top of the toilet. move to the right a bit and face the toilet again. place the bar of soap in the garbage bin to the right of the toilet. ',
 'Move a rag from a cabinet to the sink. turn around, walk a few steps, then turn right and walk over to the bathroom sinks. open the left cabinet under the right sink and take out the red rag that is inside. move over to stand directly in front of the right hand sink. put the red rag in the sink. ',
 'Inspect keys by lamp light. walk forward until you are past the bed on your right then when you are turn right and walk to the black 

In [17]:
from transformers import BertTokenizer

In [18]:
# Load pre-trained model tokenizer (vocabulary)
encoder_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# tokenized_text = encoder_tokenizer(tasks, padding=True, truncation=True, max_length=100, return_tensors="pt").input_ids

decoder_tokenizer = BasicTokenizer("yz/vocab.txt")

#targets = torch.LongTensor(decoder_tokenizer.tokenize(plans))

In [19]:
from transformers import BertConfig, EncoderDecoderConfig, EncoderDecoderModel

# 导入模型 BERT

In [20]:
# Initializing a BERT bert-base-uncased style configuration
config_encoder = BertConfig()
config_decoder = BertConfig()

config_decoder.update({
    "vocab_size": len(decoder_tokenizer.vocab),
    "num_hidden_layers":3,
    "num_attention_heads":3
})

config = EncoderDecoderConfig.from_encoder_decoder_configs(config_encoder, config_decoder)

In [21]:
model = EncoderDecoderModel(config=config)

In [25]:
model.encoder = BertModel.from_pretrained('bert-base-uncased')

In [26]:
if args.gpu and torch.cuda.is_available():
    model = model.cuda()

In [27]:
loss_fun = nn.CrossEntropyLoss(ignore_index=0)

In [28]:
optimizer = torch.optim.Adam(model.decoder.parameters(), lr=args.lr)

In [29]:
from datetime import datetime

# datetime object containing current date and time
begin_time = datetime.now()
print(begin_time)

2021-03-15 20:12:40.785960


In [ ]:
for epoch in range(args.epoch):
    model.train()
    train_loss_list = []
    train_acc_list = []
    for tasks, plans in tqdm(train_dataloader):
        tokenized_text = encoder_tokenizer(tasks, padding=True, truncation=True, 
                                           max_length=100, return_tensors="pt").input_ids
#         try:
        targets = torch.LongTensor(decoder_tokenizer.tokenize(plans))
#         except:
#             print(tasks)
#             print(plans)
        if args.gpu and torch.cuda.is_available():
            tokenized_text = tokenized_text.to("cuda")
            targets = targets.to("cuda")
        
        outputs = model(input_ids = tokenized_text, decoder_input_ids = targets[:,:-1])
        
        output_dim = outputs.logits.shape[-1]
        outputs.logits = outputs.logits.contiguous().view(-1, output_dim)
        labels = targets[:,1:].contiguous().view(-1)
        
        train_loss = loss_fun(outputs.logits, labels)
        #print(train_loss)
        
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        train_acc = calculate_accuracy(outputs.logits, labels)
        
        train_loss_list.append(train_loss.item())
        train_acc_list.append(train_acc)
        
        
    mean_train_loss = sum(train_loss_list) / (len(train_loss_list) + 1e-4)
    mean_train_acc = sum(train_acc_list) / (len(train_acc_list) + 1e-4)
    print("epoch: {} train_loss: {:.3f}, train_acc: {:.3f}".format(epoch, mean_train_loss, mean_train_acc))

In [22]:
# datetime object containing current date and time
end_time = datetime.now()
print(end_time)

2021-03-15 20:11:33.176086


In [23]:
torch.save(model.state_dict(), "exp/{}.pth".format(end_time))

In [24]:
#model.load_state_dict(torch.load("exp/2021-03-15 20:11:33.176086.pth"))

<All keys matched successfully>